# Land Use Classification and Machine Learning
## ZFL, Bonn March/April 2019

### Outline
Land cover classification of remote sensing imagery is a task which falls into the general category of pattern recognition. Pattern recognition problems, in turn, are usually approached by developing appropriate machine learning algorithms. Broadly speaking, machine learning involves tasks for which there is no known direct method to compute a desired output from a set of inputs. The strategy adopted is for the computer to learn from a set of representative examples.

In the case of supervised classification, the task can often be seen as one of modeling probability distributions.  This is called the training phase of the classification procedure. Then these probabilities are used to classify all of the pixels in the image, a step  referred to as the generalization phase.

The course will treat three representative models for supervised classification which involve probability estimation: a parametric model (the Bayes maximum likelihood classifier), a nonparametric model (Gaussian kernel classification), and a semiparametric or mixture model (the feed-forward neural network or FFN). In addition statistical methods for accuracy assessment and model comparison will be discussed.

The theory will be illustrated with Python programs for classification and evaluation, both for local processing as well as on the Google Earth Engine. In the case of neural networks, deep learning techniques with TensorFlow will be introduced.

"Deep learning is a particular kind of machine learning that achieves great power and ﬂexibility by representing the world as a nested hierarchy of concepts, with each concept deﬁned in relation to simpler concepts, and more abstract representations computed in terms of less abstract ones."

In [ ]:
import warnings
# these are innocuous but irritating
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
# enable in-line graphics
%matplotlib inline

## Training data separability
$$
\epsilon = \int\min[\ p(g\mid 1)Pr(1),p(g\mid 2)Pr(2)\ ]dg.
$$


$$
B = {1\over 8}(\mu_2-\mu_1)^\top \left[{\Sigma_1+\Sigma_2\over 2}\right]^{-1} (\mu_2-\mu_1)
+{1\over 2}\log\left({\left|\Sigma_1+\Sigma_2\right|/2\over \sqrt{|\Sigma_1||\Sigma_2|}}\right).
$$

The training regions are in ENVI shapefiles in the imagery directory:

In [ ]:
!ls imagery | grep train

and these have been uploaded to GEE as a table in the shared Asset

    users/mortcanty/supervisedclassification/train
    
together with the 9-band ASTER PCA image 

    users/mortcanty/supervisedclassification/AST_20070501_pca


In [ ]:
import ee
ee.Initialize()

# first 4 principal components of ASTER image
image = ee.Image('users/mortcanty/supervisedclassification/AST_20070501_pca') \
            .select(0,1,2,3)

# training data
table = ee.FeatureCollection('users/mortcanty/supervisedclassification/train')
trainData = image.sampleRegions(table,['CLASS_ID'])
print trainData.size().getInfo()  

A function to calculate the Jeffries-Matusita separability on the GEE

In [ ]:
def jmsep(class1,class2,image,table):
# Jeffries-Matusita separability    
    table1 = table.filter(
        ee.Filter.eq('CLASS_ID',str(class1-1)))
    m1 = image.reduceRegion(ee.Reducer.mean(),table1)\
              .toArray() 
    s1 = image.toArray() \
         .reduceRegion(ee.Reducer.covariance(),table1)\
         .toArray()
    table2 = table.filter(
        ee.Filter.eq('CLASS_ID',str(class2-1)))
    m2 = image.reduceRegion(ee.Reducer.mean(),table2)\
              .toArray()
    s2 = image.toArray() \
        .reduceRegion(ee.Reducer.covariance(),table2,15)\
              .toArray()
    m12 = m1.subtract(m2)  
    m12 = ee.Array([m12.toList()]) # makes 2D matrix  
    s12i = s1.add(s2).divide(2).matrixInverse()
#  first term in Bhattacharyya distance
    B1 = m12.matrixMultiply(
          s12i.matrixMultiply(m12.matrixTranspose())) \
            .divide(8)
    ds1 = s1.matrixDeterminant()
    ds2 = s2.matrixDeterminant() 
    ds12 = s1.add(s2).matrixDeterminant()
#  second term
    B2 = ds12.divide(2).divide(ds1.multiply(ds2).sqrt())\
             .log().divide(2)
    B = ee.Number(B1.add(B2).project([0]).toList().get(0))
#  J-M separability
    return ee.Number(1).subtract(ee.Number(1) \
             .divide(B.exp())) \
             .multiply(2)

In [ ]:
print jmsep(5,9,image,table).getInfo()

Band mean values for the training data, illustrating iteration over an ee.List object

In [ ]:
def band_mean(current,prev):
    current = ee.String(current)
    prev = ee.Dictionary(prev)
    trainData = ee.FeatureCollection(prev.get('trainData'))
    class_id = prev.get('class_id')
    means = ee.List(prev.get('means'))
    mu = trainData.filter(ee.Filter.eq('CLASS_ID',class_id)).aggregate_mean(current)
    return ee.Dictionary({ 'trainData':trainData,'class_id':class_id,'means':means.add(mu) })

def class_mean(trainData,class_id,bandNames):
    first = ee.Dictionary({'trainData':trainData,'class_id':str(class_id),'means':ee.List([])})
    return ee.Dictionary(bandNames.iterate(band_mean,first)).get('means')

mu = ee.Array(class_mean(trainData,9,image.bandNames()))
print mu.getInfo()    

## Bayes Maximum Likeliood Classifier (parametric model)

### Naive Bayes on the GEE

In [ ]:
import IPython.display as disp
jet = 'black,blue,cyan,yellow,red,brown'

# rename the class ids from strings to integers
trainData = trainData\
    .remap(['0','1','2','3','4','5','6','7','8','9'],
           [0,1,2,3,4,5,6,7,8,9],'CLASS_ID')
    
# train a naive Bayes classifier    
classifier = ee.Classifier.continuousNaiveBayes()
trained = classifier\
    .train(trainData,'CLASS_ID',image.bandNames())

# classify the image and display    
classified = image.classify(trained)
url = classified.select('classification')\
    .getThumbURL({'min':0,'max':9,'palette':jet})
disp.Image(url=url)

### Gaussian Bayes Maximum Likelihood with the classify.py script

In [ ]:
run scripts/classify -p [1,2,3,4] -a 1 imagery/AST_20070501_pca.tif imagery/train.shp

In [ ]:
run scripts/dispms -f imagery/AST_20070501_pca_class.tif -c \
-r  "['WATER', 'RAPESEED', 'SUGARBEET', 'SUBURBAN', 'INDUSTRIAL', 'CONIFEROUS', 'GRAIN', 'GRASSLAND', 'HERBIFEROUS', 'OPENCAST']" 

## Gauss Kernel Classifier (non-parametric model)

In [ ]:
run scripts/classify -p [1,2,3,4] -a 2 -P imagery/AST_20070501_pca.tif imagery/train.shp

## Feed Forward Neural Network (mixture model)

### Implementation in low-level TensorFlow
#### Network architecture

In [ ]:
import tensorflow as tf
from datetime import datetime
# placeholders
Gs = tf.placeholder(tf.float32,shape=(None,4))
ls = tf.placeholder(tf.int64,shape=(None))
# hidden layer with rectified linear units (relu) 
hidden=tf.layers.dense(Gs,10,activation=tf.nn.relu)
# output layer
logits=tf.layers.dense(hidden,10)
# cross entropy cost function
xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits\
                            (labels=ls,logits=logits)
cost=tf.reduce_mean(xentropy)
# training algorithm with 0.01 learning rate
optimizer=tf.train.GradientDescentOptimizer(0.01)
training_op=optimizer.minimize(cost)
# variables initializer 
init=tf.global_variables_initializer()
# accuracy evaluation
correct=tf.nn.in_top_k(logits,ls,1)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
# saver
saver = tf.train.Saver()
# logger
cost_summary = tf.summary.scalar('COST',cost)

#### Exporting train/test data from GEE to Google Drive for classification with TensorFlow

In [ ]:
# sample the image with the polygons to a feature  
# collection, rename the class id columns from strings to 
# integers and add a column of random numbers in [0,1]
trainTestData = trainData.randomColumn('rand',seed=12345) 
    
# filter on the random column to split into training and test
# feature collections in the ration of 2:1
trainData = trainTestData.filter(ee.Filter.lt('rand',0.67))
testData = trainTestData.filter(ee.Filter.gte('rand',0.67))

print 'train pixels: %i'%trainData.size().getInfo()
print 'test pixels:  %i'%testData.size().getInfo()    

# Export feature collections as csv files
gdexport = ee.batch.Export.table.toDrive(trainData,description='driveexporttask',folder= 'EarthEngineImages',fileNamePrefix='traindata')    
gdexport.start() 

gdexport = ee.batch.Export.table.toDrive(testData,description='driveexporttask',folder= 'EarthEngineImages',fileNamePrefix='testdata')    
gdexport.start()  

< Download the CSV files from Google Drive to myimagery folder>

In [ ]:
!ls myimagery | grep csv

Read in as Pandas dataframes

In [ ]:
import pandas as pd

dftrain = pd.read_csv('myimagery/traindata.csv')
dftest = pd.read_csv('myimagery/testdata.csv')
print dftrain.head(5)

Convert relevant columns to numpy arrays

In [ ]:
Gstrn = dftrain.values[:,2:6]
lstrn = dftrain.values[:,1]

Gstst = dftest.values[:,2:6]
lstst = dftest.values[:,1]

print Gstrn.shape
print lstrn.shape
print Gstst.shape
print lstst.shape

#### Training and testing

In [ ]:
%%time
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir = 'tf_logs/run-'+str(now)
file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
with tf.Session() as sess:
    init.run()
    for epoch in range(5000):
        if epoch % 200 ==0:
            summary_str = cost_summary.eval(feed_dict={Gs:Gstrn,ls:lstrn})
            file_writer.add_summary(summary_str,epoch)
        sess.run(training_op,feed_dict={Gs:Gstrn,ls:lstrn})
    acc = accuracy.eval(feed_dict={Gs:Gstst,ls:lstst})
    file_writer.close()
    print 'Test accuracy: %f'%acc
    save_path = saver.save(sess,'imagery/dnn.ckpt')

In [ ]:
!tensorboard --logdir tf_logs/

#### Prediction

In [ ]:
import auxil.readshp as rs
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly
import matplotlib.pyplot as plt
import numpy as np

infile='imagery/AST_20070501_pca.tif'
gdal.AllRegister()
inDataset = gdal.Open(infile,GA_ReadOnly)
# read entire image
cols = inDataset.RasterXSize
rows = inDataset.RasterYSize  
Gs_all = np.zeros((cols*rows,4))
for b in range(4):
    band = inDataset.GetRasterBand(b+1)
    Gs_all[:,b] = band.ReadAsArray(0,0,cols,rows)\
                          .astype(float).ravel()    

In [ ]:
with tf.Session() as sess:
    saver.restore(sess,'imagery/dnn.ckpt')
    Z = logits.eval(feed_dict={Gs:Gs_all})
    cls = np.argmax(Z,1)  

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.reshape(cls/10.0,(rows,cols)),cmap='jet')

### Implementation in high-level TensorFlow (keras)

In [ ]:
run scripts/classify -p [1,2,3,4] -a 6 -e 1000 -L [10,10,10] imagery/AST_20070501_pca.tif imagery/train.shp